<a href="https://colab.research.google.com/github/dnr6419/BigData_study/blob/master/dacon_universe_lightgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [47]:
os.getcwd()
data = pd.read_csv("./universe/train.csv")
data.head()
column_number = {}
cnt = 0
# 결측치 확인
data = data.dropna()

# type을 가변수 형태로 변환.
column_name = set(data['type'].values)
for i in column_name:
  column_number[i] = cnt
  cnt += 1
def to_number(x, dic):
    return dic[x]
data['type_num'] = data['type'].apply(lambda x: to_number(x, column_number)) 

{'SERENDIPITY_DISTANT': 0, 'REDDEN_STD': 1, 'GALAXY': 2, 'STAR_WHITE_DWARF': 3, 'STAR_RED_DWARF': 4, 'QSO': 5, 'SERENDIPITY_BLUE': 6, 'SERENDIPITY_FIRST': 7, 'STAR_SUB_DWARF': 8, 'SKY': 9, 'SPECTROPHOTO_STD': 10, 'STAR_CARBON': 11, 'STAR_CATY_VAR': 12, 'SERENDIPITY_RED': 13, 'ROSAT_D': 14, 'SERENDIPITY_MANUAL': 15, 'STAR_BROWN_DWARF': 16, 'STAR_PN': 17, 'STAR_BHB': 18}


In [53]:
modified_data = data.drop(['id'],axis=1)
feature_columns = list(data.columns.difference(['type','type_num'])) # target을 제외한 모든 행
X = data[feature_columns] # 설명변수
y = data['type_num'] # 타겟변수
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2) # 학습데이터와 평가데이터의 비율을 8:2 로 분할| 
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인



   fiberID  fiberMag_g  fiberMag_i  ...   psfMag_r   psfMag_u   psfMag_z
0      601   21.644453   21.387653  ...  21.314148  23.198224  21.171444
1      788   21.029773   20.937731  ...  20.678850  21.431355  20.473229
2      427   17.033098   17.095999  ...  16.679677  17.851451  16.641788
3      864   20.317165   20.073852  ...  19.926909  20.789900  19.463270
4      612   23.629122   19.861718  ...  21.471406  26.454969  18.389096

[5 rows x 22 columns]
(159992, 22) (39999, 22) (159992,) (39999,)


In [60]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
start = time.time() # 시작 시간 지정
lgb_dtrain = lgb.Dataset(data = train_x, label = train_y) # 학습 데이터를 LightGBM 모델에 맞게 변환
lgb_param = {'max_depth': 15, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 100, # Number of trees, 트리 생성 개수
            'objective': 'multiclassova', # 목적 함수
            'num_class': len(set(train_y)) + 1} # 파라미터 추가
lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행
lgb_model_predict = np.argmax(lgb_model.predict(test_x), axis = 1) # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측
print("Accuracy: %.2f" % (accuracy_score(test_y, lgb_model_predict) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Accuracy: 85.28 %
Time: 43.70 seconds
